## Imports and Downloads

In [13]:
! pip show tensorflw

In [8]:
#! pip3 install keras
#! pip3 install tensorflow
#! pip3 install Keras-Preprocessing
#! pip3 install nltk
# ! pip3 install pyyaml h5py
from tensorflow import keras
import pandas as pd
import numpy as np
import re
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import load_model
import urllib.request
import zipfile
import os
from keras.models import Sequential
from keras.layers import Embedding,Bidirectional,LSTM,GRU,Dense
import nltk
from nltk.tokenize import word_tokenize
import warnings
import tensorflow as tf
nltk.download('punkt')
warnings.filterwarnings('ignore')
print(tf.version.VERSION)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.6/197.6 kB 1.0 MB/s eta 0:00:0000:0100:01
2.11.0


[nltk_data] Downloading package punkt to /Users/y.k./nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Connecting to dataset

In [10]:
!unzip "/Users/y.k./Desktop/Downloads/Datasets_S_Cb/archive.zip"

Archive:  /Users/y.k./Desktop/Downloads/Datasets_S_Cb/archive.zip
replace test.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [11]:
f=open('train.txt','r')
x_train=[]
y_train=[]
for i in f:
    l=i.split(';')
    y_train.append(l[1].strip())
    x_train.append(l[0])
f=open('test.txt','r')
x_test=[]
y_test=[]
for i in f:
    l=i.split(';')
    y_test.append(l[1].strip())
    x_test.append(l[0])
f=open('val.txt','r')
for i in f:
    l=i.split(';')
    y_test.append(l[1].strip())
    x_test.append(l[0])
data_train=pd.DataFrame({'Text':x_train,'Emotion':y_train})
data_test=pd.DataFrame({'Text':x_test,'Emotion':y_test})
data=data_train.append(data_test,ignore_index=True)

In [12]:
def clean_text(data):
    data=re.sub(r"(#[\d\w\.]+)", '', data)
    data=re.sub(r"(@[\d\w\.]+)", '', data)
    data=word_tokenize(data)
    return data
texts=[' '.join(clean_text(text)) for text in data.Text]
texts_train=[' '.join(clean_text(text)) for text in x_train]
texts_test=[' '.join(clean_text(text)) for text in x_test]

In [13]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(texts)
sequence_train=tokenizer.texts_to_sequences(texts_train)
sequence_test=tokenizer.texts_to_sequences(texts_test)
index_of_words=tokenizer.word_index
vocab_size=len(index_of_words)+1

In [14]:
num_classes=6
embed_num_dims=300
max_seq_len=500
class_names=['anger','sadness','fear','joy','surprise','love']
X_train_pad=pad_sequences(sequence_train,maxlen=max_seq_len)
X_test_pad=pad_sequences(sequence_test,maxlen=max_seq_len)
encoding={'anger':0,'sadness':1,'fear':2,'joy':3,'surprise':4,'love':5}
y_train=[encoding[x] for x in data_train.Emotion]
y_test=[encoding[x] for x in data_test.Emotion]
y_train=to_categorical(y_train)
y_test=to_categorical(y_test)

In [22]:
!unzip "/Users/y.k./Desktop/Downloads/Datasets_S_Cb/wiki-news-300d-1M.vec.zip"

Archive:  /Users/y.k./Desktop/Downloads/Datasets_S_Cb/wiki-news-300d-1M.vec.zip
  inflating: wiki-news-300d-1M.vec   


In [15]:
def create_embedding_matrix(filepath,word_index,embedding_dim):
    vocab_size=len(word_index)+1
    embedding_matrix=np.zeros((vocab_size,embedding_dim))
    with open(filepath) as f:
        for line in f:
            word,*vector=line.split()
            if word in word_index:
                idx=word_index[word]
                embedding_matrix[idx] = np.array(vector,dtype=np.float32)[:embedding_dim]
    return embedding_matrix
fname='wiki-news-300d-1M.vec'
embedd_matrix=create_embedding_matrix(fname,index_of_words,embed_num_dims)


In [16]:
embedd_layer=Embedding(vocab_size,embed_num_dims,input_length=max_seq_len,weights=[embedd_matrix],trainable=False)
gru_output_size=128
bidirectional=True
model=Sequential()
model.add(embedd_layer)
model.add(Bidirectional(GRU(units=gru_output_size,dropout=0.2,recurrent_dropout=0.2)))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

2022-11-28 00:56:18.811945: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [17]:
batch_size=128
epochs=8
model.fit(X_train_pad,y_train,batch_size=batch_size,epochs=epochs,validation_data=(X_test_pad,y_test))

Epoch 1/8
125/125 [==============================] - 361s 3s/step - loss: 1.3809 - accuracy: 0.4835 - val_loss: 1.1150 - val_accuracy: 0.6100
Epoch 2/8
125/125 [==============================] - 376s 3s/step - loss: 0.8637 - accuracy: 0.7038 - val_loss: 0.6224 - val_accuracy: 0.7870
Epoch 3/8
125/125 [==============================] - 387s 3s/step - loss: 0.5190 - accuracy: 0.8217 - val_loss: 0.3696 - val_accuracy: 0.8748
Epoch 4/8
125/125 [==============================] - 354s 3s/step - loss: 0.3394 - accuracy: 0.8812 - val_loss: 0.2704 - val_accuracy: 0.9003
Epoch 5/8
125/125 [==============================] - 357s 3s/step - loss: 0.2555 - accuracy: 0.9078 - val_loss: 0.2236 - val_accuracy: 0.9162
Epoch 6/8
125/125 [==============================] - 375s 3s/step - loss: 0.2224 - accuracy: 0.9156 - val_loss: 0.2045 - val_accuracy: 0.9218
Epoch 7/8
125/125 [==============================] - 373s 3s/step - loss: 0.1922 - accuracy: 0.9249 - val_loss: 0.1772 - val_accuracy: 0.9283
Epoch 

In [18]:
message=['I am sad.']
seq=tokenizer.texts_to_sequences(message)
padded=pad_sequences(seq,maxlen=max_seq_len)
pred=model.predict(padded)
print('Message:'+str(message))
print('Emotion:',class_names[np.argmax(pred)])

1/1 [==============================] - 1s 1s/step
Message:['I am sad.']
Emotion: sadness


In [19]:
model.save('my_model.h5')

In [24]:
New_Model = tf.keras.models.load_model('my_model.h5')


In [ ]:
import pickle

with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [29]:
message=['I am sad.']
seq=tokenizer.texts_to_sequences(message)
padded=pad_sequences(seq,maxlen=max_seq_len)
pred=New_Model.predict(padded)
print('Message:'+str(message))
print('Emotion:',class_names[np.argmax(pred)])

1/1 [==============================] - 0s 136ms/step
Message:['I am sad.']
Emotion: sadness
